Este algoritmo é projetado para fornecer uma partição fuzzy e um protótipo para cada cluster fuzzy, além de aprender um peso de relevância para cada matriz de dissimilaridade que muda na iteração de cada algoritmo e é diferente de um cluster fuzzy para outro.

In [49]:
# PARÂMETROS INICIAIS

K = 2 # Número de classes
N = 6 # Número de objetos
t = 0 # tempo inicial
m = 1.6 # valor m 
T = 150  # Número máximo de rodadas
e = 1e-10 # Erro permissível
s = 1 # valor s 
q = 2 # cardinalidade q

In [50]:
# Dados de entrada
p1 = 3 # 3 variaveis descritivas para cada objeto
p2 = 4 # 4 variaveis descritivas para cada objeto

# Matrizes de objetos e_i normalziados
E1 = [[0.13043, 0.32609, 0.44022],
     [0.45109, 0.03804, 0.91304],
     [0.4837, 0.34783, 0.47283],
     [0.34783, 0.4837, 1.0],
     [0.49457, 0.0, 0.57609],
     [0.49457, 0.0, 0.40761]]
E2 = [[0.1005, 0.80905, 1.0, 0.8593],
      [0.31658, 0.64824, 0.46231, 0.61809],
      [0.31658, 0.1809, 0.66332, 0.84422],
      [0.79899, 0.08543, 0.78392, 0.78392],
      [0.67839, 0.71357, 0.85427, 0.73367],
      [0.73367, 0.40201, 0.0, 0.1005]]

# Matrizes de Dissimilaridade
D1 = [[0.0, 0.6398, 0.3554, 0.6208, 0.5073, 0.4898],
     [0.6398, 0.0, 0.5392, 0.4656, 0.3418, 0.5087],
     [0.3554, 0.5392, 0.0, 0.5610, 0.3629, 0.3540],
     [0.6208, 0.4656, 0.5610, 0.0, 0.6596, 0.7787],
     [0.5073, 0.3418, 0.3629, 0.6596, 0.0, 0.1684],
     [0.4898, 0.5087, 0.3540, 0.7787, 0.1684, 0.0]]
D2 = [[0.0, 0.64795, 0.74488, 1.03145, 0.61652, 1.46368],
      [0.64795, 0.0, 0.55673, 0.82487, 0.5497, 0.8463],
      [0.74488, 0.55673, 0.0, 0.50992, 0.68068, 1.10271],
      [1.03145, 0.82487, 0.50992, 0.0, 0.64543, 1.08907],
      [0.61652, 0.5497, 0.68068, 0.64543, 0.0, 1.10942],
      [1.46368, 0.8463, 1.10271, 1.08907, 1.10942, 0.0]]
# Dj = D[0] = = D1 e D[1] = D2  Matriz de dissimilaridade com "p" visões
Dj = [[[0.0, 0.6398, 0.3554, 0.6208, 0.5073, 0.4898],
      [0.6398, 0.0, 0.5392, 0.4656, 0.3418, 0.5087],
      [0.3554, 0.5392, 0.0, 0.5610, 0.3629, 0.3540],
      [0.6208, 0.4656, 0.5610, 0.0, 0.6596, 0.7787],
      [0.5073, 0.3418, 0.3629, 0.6596, 0.0, 0.1684],
      [0.4898, 0.5087, 0.3540, 0.7787, 0.1684, 0.0]],
     [[0.0, 0.64795, 0.74488, 1.03145, 0.61652, 1.46368],
      [0.64795, 0.0, 0.55673, 0.82487, 0.5497, 0.8463],
      [0.74488, 0.55673, 0.0, 0.50992, 0.68068, 1.10271],
      [1.03145, 0.82487, 0.50992, 0.0, 0.64543, 1.08907],
      [0.61652, 0.5497, 0.68068, 0.64543, 0.0, 1.10942],
      [1.46368, 0.8463, 1.10271, 1.08907, 1.10942, 0.0]]]
P = len(Dj) # Número de Partições Fuzzy
# Gk é o representante do grupo Pk (1 <= k <= K)
'''G = [[g1 = g11, g12, ..., g1p],
     [g2 = g21, g22, ..., g2p],
     [g3 = g31, g32, ..., g3p],
     [gK = gK1, gK2, ..., gKp]]
G = [[2],
     [3],
     [1]]'''

#Uik é o grau de pertinência exclusivo do objeto i no grupo k. Logo, uik = 1, se o objeto i estiver no grupo k; e 0, otherwise.
#Uik é o representante do grupo Pk (i = 1,..., N) e (k= 1,...,K)
'''U = [[u1 = u11, u12, u13],
     [u2 = u21, u22, u23],
     [u3 = u31, u32, u33],
     [u4 = u41, u42, u43],
     [u5 = u51, u52, u53],
     [u6 = u61, u62, u63]]'''
U = [[1, 0, 0],
     [0, 0, 1],
     [1, 0, 0],
     [0, 0, 1],
     [0, 1, 0],
     [0, 1, 0]]

# V é um vetor k-dimensional de vetores de peso de relevância (um para cada partição fuzzy P)
'''v = [[v1 = v11, v12, ..., v1p],
        [v2 = v21, v22, ..., v2p],
        [v3 = v31, v32, ..., v3p]]'''
v = [[1, 1],
     [1, 1],
     [1, 1]]

In [51]:
# Construir as matrizes V, U e G, a partir dos dados de entrada e a Matriz de Dissimilidaridade
import numpy as np

# Matriz V de vetores de pesos de relevância
v = np.arange(K*P)  
v = v.reshape((K, P))
v = np.ones_like(v, dtype=float) # Inicialmente é igual a 1 para todo v_kj
print('v =',v)

# Matriz U (dimensão NxK)
U = np.arange(N*K) 
U = U.reshape((N, K))
U = np.zeros_like(U, dtype=float) # Inicialmente é igual a 0 para todo U_ik
print('U =',U)

# Matriz de representantes Gk para cada cluster k
G = np.arange(K*1)  #
G = G.reshape((K, 1))
G = np.ones_like(G,  dtype=int) # Inicialmente é igual a 1 para todo Gk
print('G =',G)
G = []
for g in range(0,K):
  Gin = [1]
  G.append(Gin)
print(G)

[[1], [1]]


In [52]:
# Função de Adequação "J" (Equações: 06 e 07)

def FMatching(ki, ii): # Equação 07
  vkj1 = 0
  for j in range(0,P):
    dj1 = 0
    for e in G[ki]:
      dj1 += Dj[j][ii][e-1]
    vkj1 += dj1 * v[ki][j]
  return vkj1

def fitnessFunction(): # Equação 06
  J = 0
  for i in range(0, N):
    for j in range(0, K):
      J += ((U[i][j])**m) * FMatching(j, i)
  return J

In [53]:
# Etapa 1: Cálculo dos melhores protótipos:
# A partição Fuzzy U e o vetor de vetores de pesos de relevância V_k são fixos

#Procedimento 2.3
import numpy as np # Usar o numpy para utilizar a função "argmin"

def prototype(clt, G_verificação):
  G_out = [] # Para armazenar os prototipos para todos os K cluster
  G_in = []  # Para armazenar os argumentos internos de cada k
  for h in range(0, N):
    G_in.append(argumentos(clt, h))
  L = np.argmin(G_in, axis=0)
  while len(G_out) == 0:
    if (L+1) not in G_verificação:
      G_out.append(L+1)
    else:
      G_in[L] = 10000000000
      L = np.argmin(G_in, axis=0)
  return G_out[0]

def argumentos(ki, hi): # Essa função vai retornar o calculo dos somatórios para o k-ésimo cluster e o h-éssimo argumento.
  L = 0
  somaUVD = 0
  for i in range(0, N):
    somaVD = 0
    for j in range(0, P):
       somaVD += ((v[ki][j])**s) * Dj[j][i][hi] # Não ficou tão claro na função, se ele soma também cada matriz de dissimilaridade Dj para as p visões diferentes
    somaUVD += ((U[i][ki])**m) * somaVD
  L += somaUVD
  return L
'''
G_verificação = []  # Uma lista para armazenar todos os protótipos de G e servir para verificar se o elemento eL já está na lista
for cluster in range(0,K):
  eL = []
  while len(eL) < q:
    eL.append(prototype(cluster, G_verificação))
    if len(eL) == 1:
      G_verificação.append(eL[0])
    elif len(eL) == 2:
      G_verificação.append(eL[1])
  G[cluster] = eL'''

cluster: 0
G_in:  [1.10028, 2.38368, 1.10028, 2.72317, 2.1673999999999998, 3.41019]
L: 0
eL: [1]
G_verificação [1]
cluster: 0
G_in:  [1.10028, 2.38368, 1.10028, 2.72317, 2.1673999999999998, 3.41019]
L: 0
eL: [1, 3]
G_verificação [1, 3]
cluster: 1
G_in:  [3.0773, 2.2465, 2.50029, 3.1727999999999996, 1.2778200000000002, 1.2778200000000002]
L: 4
eL: [5]
G_verificação [1, 3, 5]
cluster: 1
G_in:  [3.0773, 2.2465, 2.50029, 3.1727999999999996, 1.2778200000000002, 1.2778200000000002]
L: 4
eL: [5, 6]
G_verificação [1, 3, 5, 6]
[[1, 3], [5, 6]]


In [55]:
#Etapa 2: Cálculo do melhor vetor de pesos de relevância das visões nos grupos (Equação 09)
# Partição Fuzzy U e vetor de representantes G fixos!

def relevanceWeights(ki, ji):
  prodUP = 1
  for h in range(0, P):
    uik2 = 0
    for i in range(0,N):
      DjUp2 = 0
      for e in G[ki]:
        DjUp2 += Dj[h][i][(e)-1]
      uik2 += ((U[i][ki])**m) * DjUp2
    prodUP *= uik2

  DjUDown2 = 0
  for i in range(0,N):
    DjDown2 = 0
    for e in G[ki]:
      DjDown2 += Dj[ji][i][(e)-1]
    DjUDown2 += ((U[i][ki])**m) * DjDown2
    
  resultado = (prodUP**(1/P))/(DjUDown2)
  return resultado
'''
for k in range(0, K):
  for j in range(0, P):
    v[k][j] = relevanceWeights(k, j) 
print(v) # A matriz de pesos de relevância depois da etapa 2'''

'\nfor k in range(0, K):\n  for j in range(0, P):\n    v[k][j] = relevanceWeights(k, j) \nprint(v) # A matriz de pesos de relevância depois da etapa 2'

In [56]:
# Etapa 3: Definição da Melhor Partição Fuzzy (Equação 11)
# Vetores de representantes G e vetores dos pesos V são fixos!

def fuzzyPartition(ii, ki, s):
  total3 = 0
  soma3 = 0
  for h in range(0,K):
    vikUP3 = 0
    for j in range(0,P):
      djUP3 = 0
      for e in G[ki]:
        djUP3 += Dj[j][ii][(e)-1]
      vikUP3 += (v[ki][j]**s) * djUP3
    vikDown3 = 0
    for j in range(0,P):
      djDown3 = 0
      for e in G[h]:
        djDown3 += Dj[j][ii][(e)-1]
      vikDown3 += (v[h][j]**s) * djDown3
    if vikDown3 == 0:
      soma3 += 0
    else:
      soma3 += (vikUP3/vikDown3)**(1/(m-1))
  if soma3 == 0:
    total3 = 0
  else:
    total3 = (soma3**(-1))
  return total3
'''
for i in range(0,N):
  for k in range(0,K):
    U[i][k] = fuzzyPartition(i, k, s)
print(U) # A matriz com a Partição Fuzzy depois da etapa 3'''

'\nfor i in range(0,N):\n  for k in range(0,K):\n    U[i][k] = fuzzyPartition(i, k, s)\nprint(U) # A matriz com a Partição Fuzzy depois da etapa 3'

In [57]:
# Algoritmo, união de todas as etapas:

#1) Inicialização
Jt = [10000] # A lista de soluções J inicia com números com diferença bem alta
t = 0 # tempo inicial
T = 50
erro = 1e-03
s = 1
# Para o grau de pertinência Uik inicial.
for i in range(0,N):
  for k in range(0,K):
    U[i][k] = fuzzyPartition(i, k, s)
print(U) # A matriz com a Partição Fuzzy calculada na Inicialização

Jt.append(fitnessFunction()) # Etapa Inicial para o J(0)
print("J('t=0')", Jt[-1])

while (t < T):  # or (abs(Jt[-1]-Jt[-2])>erro)
  t += 1
  #2) Etapa 1:
  G_verificação = []  # Uma lista para armazenar todos os protótipos de G e servir para verificar se o elemento eL já está na lista
  for cluster in range(0,K):
    eL = []
    while len(eL) < q:
      eL.append(prototype(cluster, G_verificação))
      if len(eL) == 1:
        G_verificação.append(eL[0])
      elif len(eL) == 2:
        G_verificação.append(eL[1])
    G[cluster] = eL
  print(G)
  #3) Etapa 2:
  for k in range(0, K):
    for j in range(0, P):
      v[k][j] = relevanceWeights(k, j) 
  
  #4) Etapa 3:
  for i in range(0,N):
    for k in range(0,K):
      U[i][k] = fuzzyPartition(i, k, s)

  #5) Critério de parada:
  Jt.append(fitnessFunction())
  print("ERRO", abs(Jt[-1]-Jt[-2]))
  print("t = %.f e J(t) = %.10f" %(t, Jt[-1]))
#print("J: ",Jt[-1])
#print("U = ",U)
#print("V = ",v)

[[0.847373330396868, 0.15262666960313193, 0], [0.47532337194785623, 0.5246766280521438, 1], [0.7970681627620457, 0.20293183723795424, 0], [0.5633321037432314, 0.43666789625676855, 1], [0.29304904284274697, 0.706950957157253, 0], [0.1630072342822369, 0.836992765717763, 0]]
J('t=0') 7.598186353414489
G_in:  [2.081720541493523, 2.4650418492535335, 1.8314955047698884, 2.6908558040652566, 2.4505481320525075, 3.8492935455199686]
L: 2
G_in:  [2.081720541493523, 2.4650418492535335, 1.8314955047698884, 2.6908558040652566, 2.4505481320525075, 3.8492935455199686]
L: 2
G_in:  [3.0981619502699598, 2.0229364161147645, 2.4242584808084935, 2.779194489501767, 1.7623638717596886, 1.922606435391389]
L: 4
G_in:  [3.0981619502699598, 2.0229364161147645, 2.4242584808084935, 2.779194489501767, 1.7623638717596886, 1.922606435391389]
L: 4
[[3, 1], [5, 6]]
ERRO 0.5199124992141231
t = 1 e J(t) = 7.0782738542
G_in:  [1.9426414681941337, 2.4280844189856037, 1.7354128852383568, 2.5199162933650663, 2.417550938952224

In [ ]:
'''
t = 1 e J(t) = 3.8020358709
U =  [[0.33508185 0.26876941 0.39614874]
 [0.60324418 0.         0.39675582]
 [0.54754497 0.45245503 0.        ]
 [0.2749835  0.32815043 0.39686607]
 [0.         0.5581375  0.4418625 ]
 [0.43021655 0.28664476 0.28313868]]
V =  [[1.39559794 0.71653875]
 [1.22356707 0.81728254]
 [1.25237983 0.79847981]]
 
[[3], [5], [2]]
ERRO 0.0006877680441688128
t = 2 e J(t) = 3.8027236390
U =  [[0.39330432 0.33846492 0.26823076]
 [0.39541961 0.60458039 0.        ]
 [0.         0.54624366 0.45375634]
 [0.39447719 0.28026424 0.32525857]
 [0.44213095 0.         0.55786905]
 [0.28878633 0.4192207  0.29199297]]
V =  [[1.25149747 0.79904277]
 [1.32851908 0.75271783]
 [1.20377986 0.83071667]]
t = 2 e J(t) = 3.8112235844
U =  [[0.3388236734540882, 0.3926795685343371, 0.26849675801157474], [0.6115105231353645, 0.3884894768646354, 0], [0.5412178299685242, 0, 0.45878217003147587], [0.2905644495352978, 0.385218125767447, 0.3242174246972552], [0, 0.4458533742634611, 0.5541466257365388], [0.402001699450491, 0.3040656641382036, 0.29393263641130535]]
V =  [[1.2519972205264718, 0.7987238179166999], [1.3410042935165782, 0.745709767548658], [1.1306895682715963, 0.8844160484549515]]
[[2], [5], [3]]



[[2], [5], [3]]
ERRO 3.4226785849966745e-09
t = 10 e J(t) = 3.8029715994
U =  [[0.27138242 0.33683816 0.39177942]
 [0.         0.60444486 0.39555514]
 [0.45727955 0.54272045 0.        ]
 [0.32236007 0.28112249 0.39651744]
 [0.55574161 0.         0.44425839]
 [0.28943828 0.42140802 0.2891537 ]]
V =  [[1.13036077 0.88467331]
 [1.3333547  0.74998798]
 [1.25049207 0.7996852 ]]
[[5], [2], [3]]
ERRO 2.7684361469937357e-09
t = 10 e J(t) = 3.8029715995
U =  [[0.3368381620399342, 0.27138241885538955, 0.3917794191046762], [0.6044448598576252, 0, 0.3955551401423748], [0.5427204539117575, 0.4572795460882425, 0], [0.2811224880002577, 0.3223600721161612, 0.3965174398835812], [0, 0.5557416078476989, 0.44425839215230106], [0.4214080154296302, 0.2894382829439469, 0.28915370162642284]]
V =  [[1.3333546937148955, 0.7499879849778553], [1.130360771132606, 0.8846733056721471], [1.250492071780073, 0.7996851979849036]]

[[5], [2], [3]]
ERRO 2.2637314245343987e-10
t = 11 e J(t) = 3.8029715996
U =  [[0.33683816 0.27138242 0.39177942]
 [0.60444486 0.         0.39555514]
 [0.54272045 0.45727955 0.        ]
 [0.28112249 0.32236007 0.39651744]
 [0.         0.55574161 0.44425839]
 [0.42140801 0.28943828 0.2891537 ]]
V =  [[1.33335469 0.74998799]
 [1.13036077 0.8846733 ]
 [1.25049207 0.7996852 ]]
[[2], [5], [3]]
ERRO 1.6007772885018312e-10
t = 11 e J(t) = 3.8029715996
U =  [[0.2713824187841223, 0.3368381625526888, 0.39177941866318894], [0, 0.6044448595352756, 0.3955551404647244], [0.4572795460558287, 0.5427204539441713, 0], [0.32236007174272596, 0.28112248814835, 0.396517440108924], [0.5557416081254695, 0, 0.4442583918745305], [0.28943828379339115, 0.4214080146599037, 0.28915370154670517]]
V =  [[1.130360773261558, 0.8846733040059297], [1.3333546876631013, 0.7499879883818805], [1.2504920654469618, 0.7996852020349056]]
'''

'\nt = 1 e J(t) = 3.8020358709\nU =  [[0.33508185 0.26876941 0.39614874]\n [0.60324418 0.         0.39675582]\n [0.54754497 0.45245503 0.        ]\n [0.2749835  0.32815043 0.39686607]\n [0.         0.5581375  0.4418625 ]\n [0.43021655 0.28664476 0.28313868]]\nV =  [[1.39559794 0.71653875]\n [1.22356707 0.81728254]\n [1.25237983 0.79847981]]\n \n[[3], [5], [2]]\nERRO 0.0006877680441688128\nt = 2 e J(t) = 3.8027236390\nU =  [[0.39330432 0.33846492 0.26823076]\n [0.39541961 0.60458039 0.        ]\n [0.         0.54624366 0.45375634]\n [0.39447719 0.28026424 0.32525857]\n [0.44213095 0.         0.55786905]\n [0.28878633 0.4192207  0.29199297]]\nV =  [[1.25149747 0.79904277]\n [1.32851908 0.75271783]\n [1.20377986 0.83071667]]\nt = 2 e J(t) = 3.8112235844\nU =  [[0.3388236734540882, 0.3926795685343371, 0.26849675801157474], [0.6115105231353645, 0.3884894768646354, 0], [0.5412178299685242, 0, 0.45878217003147587], [0.2905644495352978, 0.385218125767447, 0.3242174246972552], [0, 0.44585337426